# USB摄像头采集迷宫图像

In [ ]:
from pynq.overlays.base import BaseOverlay
from pynq.lib.video import *
base = BaseOverlay("base.bit")
switches = base.switches[1]
# monitor configuration: 640*480 @ 60Hz
Mode = VideoMode(640,480,24)
hdmi_out = base.video.hdmi_out
hdmi_out.configure(Mode,PIXEL_BGR)
hdmi_out.start()
print("HDMI Initialized")

# monitor (output) frame buffer size
frame_out_w = 1600
frame_out_h = 900
# camera (input) configuration
frame_in_w = 640
frame_in_h = 480

# initialize camera from OpenCV
import cv2
videoIn = cv2.VideoCapture(-1)
videoIn.set(cv2.CAP_PROP_FRAME_WIDTH, frame_in_w);
videoIn.set(cv2.CAP_PROP_FRAME_HEIGHT, frame_in_h);

print("Capture device is open: " + str(videoIn.isOpened()))

# Capture webcam image

while True:
    ret, frame_vga = videoIn.read()
   
    # display webcam image via HDMI Out
    if (ret):
        outframe = hdmi_out.newframe()
        outframe[0:480,0:640,:] = frame_vga[0:480,0:640,:]
        hdmi_out.writeframe(outframe)   
    else:
        raise RuntimeError("Failed to read from camera.")
        break
    if switches.read():
        cv2.imwrite("test.png", frame_vga)
        break            
videoIn.release()
hdmi_out.stop()
del hdmi_out

# 图像裁剪，提取最边缘的点

In [ ]:
import cv2
import numpy as np
image = cv2.imread('test.png')
image = cv2.resize(image,(640,480))
img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
ret, binary = cv2.threshold(img, 135, 255, cv2.THRESH_BINARY)
a=[]
b=[]
c=[]
d=[]
for x in range(binary.shape[0]):   # 图片的高
    for y in range(binary.shape[1]):   # 图片的宽
        px = binary[x,y]
        if(px==0):
            a.append([x,y])
            break
for x in range(binary.shape[0]):   # 图片的高
    for y in range(binary.shape[1]):   # 图片的宽
        px = binary[x,(binary.shape[1]-y-1)]
        if(px==0):
            b.append([x,(binary.shape[1]-y-1)])
            break           
for y in range(binary.shape[1]):   # 图片的宽
    for x in range(binary.shape[0]):   # 图片的高
        px = binary[x,y]
        if(px==0):
            c.append([x,y])
            break   
            
for y in range(binary.shape[1]):   # 图片的宽
    for x in range(binary.shape[0]):   # 图片的高
        px = binary[(binary.shape[0]-x-1),y]
        if(px==0):
            d.append([(binary.shape[0]-x-1),y])
            break               
a.extend(b)
a.extend(c)
a.extend(d)
max_x=0
max_y=0
min_x=10000
min_y=10000
for i in range(len(a)):
    if(a[i][0]>max_x):
        max_x=a[i][0]
    if(a[i][0]<min_x):
        min_x=a[i][0]
    if(a[i][1]>max_y):
        max_y=a[i][1]
    if(a[i][1]<min_y):
        min_y=a[i][1]        
I=np.ones((binary.shape[0],binary.shape[1]),dtype=np.uint8) 
I=I*255
for i in range(len(a)):   
    I[a[i][0]][a[i][1]]=0  
image=image[min_x:max_x+2,min_y:max_y+2]
I=I[min_x:max_x+2,min_y:max_y+2]
cv2.imwrite("img_1.jpg",image)

# 显示裁剪后的图像

In [ ]:
from pynq.overlays.base import BaseOverlay
from pynq.lib.video import *
import cv2
base = BaseOverlay("base.bit")
switches = base.switches[1]
# monitor configuration: 640*480 @ 60Hz
Mode = VideoMode(640,480,24)
hdmi_out = base.video.hdmi_out
hdmi_out.configure(Mode,PIXEL_BGR)
hdmi_out.start()
# monitor (output) frame buffer size
frame_out_w = 1600
frame_out_h = 900
test = cv2.imread('./img_1.jpg')
test=cv2.resize(test,(640,480))
outframe = hdmi_out.newframe()
outframe[:] = test
hdmi_out.writeframe(outframe)
while True:
    if switches.read():
        break
hdmi_out.stop()
del hdmi_out

# 图像取反

In [ ]:
height, width = I.shape
for row in range(height):
    for list in range(width):
        pv = I[row, list]
        I[row, list] = 255 - pv
cv2.imwrite('I.png',I)

# 图像膨胀

In [ ]:
 ## b.设置卷积核5*5
kernel = np.ones((5,5),np.uint8)
 
## c.图像的腐蚀，默认迭代次数
erosion = cv2.dilate(I,kernel)
 
## 腐蚀后
cv2.imwrite('processed.png',erosion)

# 显示膨胀后的图像

In [ ]:
from pynq.overlays.base import BaseOverlay
from pynq.lib.video import *
import cv2
base = BaseOverlay("base.bit")
switches = base.switches[1]
# monitor configuration: 640*480 @ 60Hz
Mode = VideoMode(640,480,24)
hdmi_out = base.video.hdmi_out
hdmi_out.configure(Mode,PIXEL_BGR)
hdmi_out.start()
# monitor (output) frame buffer size
frame_out_w = 1600
frame_out_h = 900
test = cv2.imread('./'processed.png')
test=cv2.resize(test,(640,480))
outframe = hdmi_out.newframe()
outframe[:] = test
hdmi_out.writeframe(outframe)
while True:
    if switches.read():
        break
hdmi_out.stop()
del hdmi_out

# 填充孔洞

In [ ]:
#python-opencv获取二值图像轮廓及中心点坐标代码：

groundtruth = cv2.imread('processed.png',0)
h1, w1 = groundtruth.shape
contours, cnt = cv2.findContours(groundtruth.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
area = []
 
# 找到最大的两个轮廓返回索引
for k in range(len(contours)):
    area.append(cv2.contourArea(contours[k]))
max_idx = np.argmax(np.array(area))  
max_area = 0
third_area=0
a=0
t=0
print(len(area))
for i in range(len(area)):
    if(area[i]>max_area and i!=max_idx):
           max_area = area[i]
           a=i
for s in range(len(area)):
    if(area[s]>third_area and s!=max_idx and s!=a):
        third_area=area[s]
for p in range(20):
    if(3.14*p*p>third_area):
        t=p
        break
print(max_area,a)     
m=[]

for j in range(len(contours)):
    if(j!=max_idx and j!=a):
        M = cv2.moments(contours[j])
        center_x = int(M["m10"] / M["m00"])
        center_y = int(M["m01"] / M["m00"])
        cv2.circle(groundtruth, (center_x, center_y), t+5, 0, -1)#绘制中心点
contours1, cnt1 = cv2.findContours(groundtruth.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
if len(contours1)>2:
    for j in range(len(contours)):
        if(j!=max_idx and j!=a):
            M = cv2.moments(contours[j])
            center_x = int(M["m10"] / M["m00"])
            center_y = int(M["m01"] / M["m00"])
            cv2.circle(groundtruth, (center_x, center_y), t+10, 0, -1)#绘制中心点
cv2.imwrite('tiandong.jpg',groundtruth)

# 显示填充孔洞后的图像

In [ ]:
from pynq.overlays.base import BaseOverlay
from pynq.lib.video import *
import cv2
base = BaseOverlay("base.bit")
switches = base.switches[1]
# monitor configuration: 640*480 @ 60Hz
Mode = VideoMode(640,480,24)
hdmi_out = base.video.hdmi_out
hdmi_out.configure(Mode,PIXEL_BGR)
hdmi_out.start()
# monitor (output) frame buffer size
frame_out_w = 1600
frame_out_h = 900
test = cv2.imread('./tiandong.jpg')
test=cv2.resize(test,(640,480))
outframe = hdmi_out.newframe()
outframe[:] = test
hdmi_out.writeframe(outframe)
while True:
    if switches.read():
        break
hdmi_out.stop()
del hdmi_out

# 图像取反

In [ ]:
height, width = groundtruth.shape
for row in range(height):
    for list in range(width):
        pv = groundtruth[row, list]
        groundtruth[row, list] = 255 - pv
cv2.imwrite('groundtruth.jpg',groundtruth)

# 骨架提取

In [ ]:
import cv2
import numpy as np
def VThin(image,array):
    h ,w= image.shape
    
    NEXT = 1
    for i in range(h):
        for j in range(w):
            if NEXT == 0:
                NEXT = 1
            else:
                M = image[i,j-1]+image[i,j]+image[i,j+1] if 0<j<w-1 else 1
                if image[i,j] == 0  and M != 0:  #上下有白点                
                    a = [0]*9
                    for k in range(3):
                        for l in range(3):
                            if -1<(i-1+k)<h and -1<(j-1+l)<w and image[i-1+k,j-1+l]==255:
                                a[k*3+l] = 1
                    sum = a[0]*1+a[1]*2+a[2]*4+a[3]*8+a[5]*16+a[6]*32+a[7]*64+a[8]*128
                    image[i,j] = array[sum]*255
                    if array[sum] == 1:#发现删除的点后，跳过右点
                        NEXT = 0
    return image
    
def HThin(image,array):
    h ,w= image.shape
    NEXT = 1
    for j in range(w):
        for i in range(h):
            if NEXT == 0:
                NEXT = 1
            else:
                M = image[i-1,j]+image[i,j]+image[i+1,j] if 0<i<h-1 else 1   
                if image[i,j] == 0 and M != 0:           #左右有白点       
                    a = [0]*9
                    for k in range(3):
                        for l in range(3):
                            if -1<(i-1+k)<h and -1<(j-1+l)<w and image[i-1+k,j-1+l]==255:
                                a[k*3+l] = 1
                    sum = a[0]*1+a[1]*2+a[2]*4+a[3]*8+a[5]*16+a[6]*32+a[7]*64+a[8]*128
                    image[i,j] = array[sum]*255
                    if array[sum] == 1:
                        NEXT = 0
    return image
    
def Xihua(image,array,num=10):
    
    iXihua = image
    for i in range(num):#搜寻的次数，不是说循环的终点，可以自己设置
        VThin(iXihua,array)#iXihua=VThin(iXihua,array)
        HThin(iXihua,array)# iXihua=HThin(iXihua,array),
    return iXihua


array = [0,0,1,1,0,0,1,1,1,1,0,1,1,1,0,1,\
         1,1,0,0,1,1,1,1,0,0,0,0,0,0,0,1,\
         0,0,1,1,0,0,1,1,1,1,0,1,1,1,0,1,\
         1,1,0,0,1,1,1,1,0,0,0,0,0,0,0,1,\
         1,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,\
         0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,\
         1,1,0,0,1,1,0,0,1,1,0,1,1,1,0,1,\
         0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,\
         0,0,1,1,0,0,1,1,1,1,0,1,1,1,0,1,\
         1,1,0,0,1,1,1,1,0,0,0,0,0,0,0,1,\
         0,0,1,1,0,0,1,1,1,1,0,1,1,1,0,1,\
         1,1,0,0,1,1,1,1,0,0,0,0,0,0,0,0,\
         1,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,\
         1,1,0,0,1,1,1,1,0,0,0,0,0,0,0,0,\
         1,1,0,0,1,1,0,0,1,1,0,1,1,1,0,0,\
         1,1,0,0,1,1,1,0,1,1,0,0,1,0,0,0]
         

image = cv2.imread('groundtruth.jpg',1)#读去自己的图
gray=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)#灰度化
ret1, th1 = cv2.threshold(gray, 100, 255, cv2.THRESH_OTSU)  #方法选择为#THRESH_OTSU
iTwo =th1#二值图像
iThin = Xihua(iTwo,array)
cv2.imwrite('gujia.jpg',iThin)

# 显示提取骨架后的图像

In [ ]:
from pynq.overlays.base import BaseOverlay
from pynq.lib.video import *
import cv2
base = BaseOverlay("base.bit")
switches = base.switches[1]
# monitor configuration: 640*480 @ 60Hz
Mode = VideoMode(640,480,24)
hdmi_out = base.video.hdmi_out
hdmi_out.configure(Mode,PIXEL_BGR)
hdmi_out.start()
# monitor (output) frame buffer size
frame_out_w = 1600
frame_out_h = 900
test = cv2.imread('./gujia.jpg')
test=cv2.resize(test,(640,480))
outframe = hdmi_out.newframe()
outframe[:] = test
hdmi_out.writeframe(outframe)
while True:
    if switches.read():
        break
hdmi_out.stop()
del hdmi_out

# 图像取反

In [ ]:
height, width = iThin.shape
for row in range(height):
    for list in range(width):
        pv = iThin[row, list]
        iThin[row, list] = 255 - pv

# 卷积核操作

In [ ]:
kernel = np.array((
        [1, 1, 1],
        [1, 10, 1],
        [1, 1, 1]), dtype="int8")
processed = cv2.filter2D(iThin/255,-1,kernel) 
cv2.imwrite('juanji.jpg',groundtruth)

# 显示卷积操作后的图像

In [ ]:
from pynq.overlays.base import BaseOverlay
from pynq.lib.video import *
import cv2
base = BaseOverlay("base.bit")
switches = base.switches[1]
# monitor configuration: 640*480 @ 60Hz
Mode = VideoMode(640,480,24)
hdmi_out = base.video.hdmi_out
hdmi_out.configure(Mode,PIXEL_BGR)
hdmi_out.start()
# monitor (output) frame buffer size
frame_out_w = 1600
frame_out_h = 900
test = cv2.imread('./juanji.jpg')
test=cv2.resize(test,(640,480))
outframe = hdmi_out.newframe()
outframe[:] = test
hdmi_out.writeframe(outframe)
while True:
    if switches.read():
        break
hdmi_out.stop()
del hdmi_out

# 提取边缘像素点

In [ ]:
m=[]
for x in range(processed.shape[0]):   # 图片的高
    for y in range(processed.shape[1]):   # 图片的宽
        px = processed[x,y]
        if(px==11):
            m.append([x,y])
            break

# 求起点和终点

In [ ]:
image = cv2.imread('img_1.jpg')
img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
ret, binary = cv2.threshold(img, 150, 255, cv2.THRESH_BINARY)
c,d=binary.shape
def juli(a,b):
    distance=((a[0]-b[0])**2+(a[1]-b[1])**2)**0.5
    return distance
def xielv(m,n):
    k=((m[1]-n[1])/(m[0]-n[0]))
    return k
def location(m,n):
    if(int((m[0]+n[0])/2)>int(c/2) and int((m[1]+n[1])/2)>int(d/2)):
        return 1
    if(int((m[0]+n[0])/2)<int(c/2) and int((m[1]+n[1])/2)>int(d/2)):
        return 2
    if(int((m[0]+n[0])/2)<int(c/2) and int((m[1]+n[1])/2)<int(d/2)):
        return 3
    if(int((m[0]+n[0])/2)>int(c/2) and int((m[1]+n[1])/2)<int(d/2)):
        return 4
    if(int((m[0]+n[0])/2)>int(c/2) and int((m[1]+n[1])/2)==int(d/2)):#在y轴上
        return 5
    if(int((m[0]+n[0])/2)<int(c/2) and int((m[1]+n[1])/2)==int(d/2)):
        return 6
    if(int((m[0]+n[0])/2)==int(c/2) and int((m[1]+n[1])/2)>int(d/2)):#在z轴上
        return 7
    if(int((m[0]+n[0])/2)==int(c/2) and int((m[1]+n[1])/2)<int(d/2)):
        return 8    
def suojin(m,n):
    a=int((m[0]+n[0])/2)
    b=int((m[1]+n[1])/2)
    point=[]
    if(xielv(m,n)>-0.1 and xielv(m,n)<0.1 and location(m,n)==1):#斜率大于-1小于1处于第一象限
        point=[a,b-5]
    if(xielv(m,n)>-0.1 and xielv(m,n)<0.1 and location(m,n)==2):#斜率大于-1小于1
        point=[a,b-5]
    if(xielv(m,n)>-0.1 and xielv(m,n)<0.1 and location(m,n)==3):#斜率大于-1小于1
        point=[a,b+5]
    if(xielv(m,n)>-0.1 and xielv(m,n)<0.1 and location(m,n)==4):#斜率大于-1小于1
        point=[a,b+5] 
    if(xielv(m,n)>-0.1 and xielv(m,n)<0.1 and location(m,n)==5):#斜率大于-1小于1
        point=[a-5,b] 
    if(xielv(m,n)>-0.1 and xielv(m,n)<0.1 and location(m,n)==6):#斜率大于-1小于1
        point=[a+5,b]
    if(xielv(m,n)>-0.1 and xielv(m,n)<0.1 and location(m,n)==7):#斜率大于-1小于1
        point=[a,b-5] 
    if(xielv(m,n)>-0.1 and xielv(m,n)<0.1 and location(m,n)==8):#斜率大于-1小于1
        point=[a,b+5]        
    if(xielv(m,n)<-0.1 and xielv(m,n)>-20 and location(m,n)==1):#斜率小于-1大于-1000
        point=[a-5,int(b-5/k)]
    if(xielv(m,n)<-0.1 and xielv(m,n)>-20 and location(m,n)==2):#斜率小于-1大于-1000
        point=[a-5,int(b-5/k)]
    if(xielv(m,n)<-0.1 and xielv(m,n)>-20 and location(m,n)==3):#斜率小于-1大于-1000
        point=[a+5,int(b+5/k)]
    if(xielv(m,n)<-0.1 and xielv(m,n)>-20 and location(m,n)==4):#斜率小于-1大于-1000
        point=[a+5,int(b+5/k)]
    if(xielv(m,n)<20 and xielv(m,n)>0.1 and location(m,n)==1):#斜率小于1000大于1
        point=[a+5,int(b+5/k)]
    if(xielv(m,n)<20 and xielv(m,n)>0.1 and location(m,n)==2):#斜率小于1000大于1
        point=[a+5,int(b+5/k)]
    if(xielv(m,n)<20 and xielv(m,n)>0.1 and location(m,n)==3):#斜率小于1000大于1
        point=[a-5,int(b-5/k)]
    if(xielv(m,n)<20 and xielv(m,n)>0.1 and location(m,n)==4):#斜率小于20大于1
        point=[a-5,int(b-5/k)]
    if((xielv(m,n)>20 or xielv(m,n)<-20) and location(m,n)==1):#斜率大于20小于-20           
        point=[a-5,b]        
    if((xielv(m,n)>20 or xielv(m,n)<-20) and location(m,n)==2):#斜率大于20小于-20            
        point=[a+5,b] 
    if((xielv(m,n)>20 or xielv(m,n)<-20) and location(m,n)==3):#斜率大于20小于-20            
        point=[a+5,b] 
    if((xielv(m,n)>20 or xielv(m,n)<-20) and location(m,n)==4):#斜率大于20小于-20           
        point=[a-5,b] 
    if((xielv(m,n)>20 or xielv(m,n)<-20) and location(m,n)==5):#斜率大于20小于-20            
        point=[a-5,b] 
    if((xielv(m,n)>20 or xielv(m,n)<-20) and location(m,n)==6):#斜率大于20小于-20           
        point=[a+5,b]     
    if((xielv(m,n)>20 or xielv(m,n)<-20) and location(m,n)==7):#斜率大于20小于-20           
        point=[a+5,b] 
    if((xielv(m,n)>20 or xielv(m,n)<-20) and location(m,n)==8):#斜率大于20小于-20           
        point=[a-5,b]        
    return point
def onepoint(m):
    a=m[0]
    b=m[1]
    if(a+5<c and a-5>0):
        if(binary[a+5][b]==0 and (location(m,m)==1 or location(m,m)==2) and binary[a-5][b]==255):
            point=[a-5,b-5]
        if(binary[a-5][b]==0 and (location(m,m)==1 or location(m,m)==2)and binary[a+5][b]==255):
            point=[a+5,b-5]
        if(binary[a+5][b]==0 and (location(m,m)==3 or location(m,m)==4) and binary[a-5][b]==255 ):
            point=[a-5,b+5]
        if(binary[a-5][b]==0 and (location(m,m)==3 or location(m,m)==4) and binary[a+5][b]==255):
            point=[a+5,b+5] 
    if(b+5<d and b-5>0):
        if(binary[a][b+5]==0 and (location(m,m)==1 or location(m,m)==4) and binary[a][b-5]==255):
            point=[a-5,b-5]
        if(binary[a][b-5]==0 and (location(m,m)==1 or location(m,m)==4)and binary[a][b+5]==255):
            point=[a-5,b+5]
        if(binary[a][b+5]==0 and (location(m,m)==3 or location(m,m)==2) and binary[a][b-5]==255):
            point=[a+5,b-5]
        if(binary[a][b-5]==0 and (location(m,m)==3 or location(m,m)==2) and binary[a][b+5]==255):
            point=[a+5,b+5]        
    return point
count=0
if len(m)==5:
    if(juli(m[0],m[1])<15):
        del m[0]
    if(juli(m[0],m[2])<15):
        del m[0]
    if(juli(m[0],m[3])<15):
        del m[0]
    if(juli(m[0],m[4])<15):
        del m[0]
    if(juli(m[1],m[2])<15):
        del m[1]
    if(juli(m[1],m[3])<15):
        del m[1]
    if(juli(m[1],m[4])<15):
        del m[1]
    if(juli(m[2],m[3])<15):
        del m[2]
    if(juli(m[2],m[4])<15):
        del m[2]
    if(juli(m[3],m[4])<15):
        del m[3]


if len(m)==2:
    start_point=onepoint(m[0])
    goal=onepoint(m[1])
if len(m)==3:
    if(juli(m[0],m[1])<20):
        start_point=suojin(m[0],m[1])
        goal=onepoint(m[2])
    if(juli(m[0],m[2])<20):
        start_point=suojin(m[0],m[2])
        goal=onepoint(m[1])
    if(juli(m[1],m[2])<20):
        start_point=suojin(m[0],m[3])
        goal=onepoint(m[0])    
if len(m)==4:
    if(juli(m[0],m[1])<50):
        start_point=suojin(m[0],m[1])
        goal=suojin(m[2],m[3])                
    elif(juli(m[0],m[2])<50):
        start_point=suojin(m[0],m[2])
        goal=suojin(m[1],m[3])                
       
    elif(juli(m[0],m[3])<50):
        start_point=suojin(m[0],m[3])
        goal=suojin(m[1],m[2])  
#start_point=tuple(start_point)
#goal=tuple(goal) 
start_point=tuple(reversed(start_point))
goal=tuple(reversed(goal))       
cv2.circle(image,start_point,3,(255,255,0),-1)
cv2.circle(image,goal,3,(255,0,255),-1)
cv2.imwrite('start_end.jpg',image)

# 显示起点和终点

In [ ]:
from pynq.overlays.base import BaseOverlay
from pynq.lib.video import *
import cv2
base = BaseOverlay("base.bit")
switches = base.switches[1]
# monitor configuration: 640*480 @ 60Hz
Mode = VideoMode(640,480,24)
hdmi_out = base.video.hdmi_out
hdmi_out.configure(Mode,PIXEL_BGR)
hdmi_out.start()
# monitor (output) frame buffer size
frame_out_w = 1600
frame_out_h = 900
test = cv2.imread('./start_end.jpg')
test=cv2.resize(test,(640,480))
outframe = hdmi_out.newframe()
outframe[:] = test
hdmi_out.writeframe(outframe)
while True:
    if switches.read():
        break
hdmi_out.stop()
del hdmi_out

# BFS解迷宫

In [ ]:
import cv2
import numpy as np


def extractGraph(input):
    kernel = np.ones((3, 3), np.uint8)
    img = imgFile.copy()
    erosion = cv2.erode(img, kernel, iterations=2)

    GRAY_img = cv2.cvtColor(erosion, cv2.COLOR_BGR2GRAY)

    (thresh, BW_img) = cv2.threshold(GRAY_img, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

    return BW_img


def BFS(graph,start,end):
    start = start[1],start[0]
    end=end[1],end[0]

    height,width = graph.shape
    levels = (np.zeros(shape=(height, width))) - 1
    parentPointerX = np.ones(shape=(height, width)) * -1
    parentPointerY = np.ones(shape=(height, width)) * -1

    def isWall(graph, point):
        if (graph[point[0], point[1]]) < 50:
            return True
        return False

    def setParent(Target,point):
        parentPointerX[Target[0]][Target[1]] = point[0]
        parentPointerY[Target[0]][Target[1]] = point[1]

    def getParent(point):
        x,y = (parentPointerX[point[0]][point[1]]),(parentPointerY[point[0]][point[1]])
        return x,y

    Frontier = []
    #将起点放到表头
    Frontier.append(start)
    levels[start[0]][start[1]] = 0
    setParent(start,(-10,-10))      

    TotalNodes = 0
    FOUND_DESTINATION = False
    while True:
        #准备更新nextFrontier
        nextFrontier = []
        #访问所有节点
        for x,y in Frontier:        #循环遍历所有节点
            #检查邻接点
            neighbours = [(x,y-1) , (x-1,y) , (x,y+1) , (x+1,y),(x-1,y-1),(x+1,y+1),(x-1,y+1),(x+1,y-1)] 

            currentLevel = levels[x][y]


            for neighbour in neighbours:                            #检查东西南北对角线是否是必要的
                tx, ty = neighbour
                if(tx<5 or ty<5 or tx>(height-5) or ty>(width-5)):  #检查图形约束
                    pass
                else:
                    TotalNodes+=1
                    if(isWall(graph,neighbour)):                        
                        pass
                    elif(levels[tx][ty] == -1):        # 设置邻接点到Next-Frontier List中和设置父节点
                        levels[tx][ty] = currentLevel + 1
                        nextFrontier.append(neighbour)
                        setParent((tx,ty),(x,y))
                    elif (levels[tx][ty] > currentLevel + 1):        #找到最短路径并更新父节点
                        levels[tx][ty] = currentLevel + 1
                        nextFrontier.append(neighbour)
                        setParent((tx, ty), (x, y))
                    else:                                            #找到重复节点跳过
                        pass

                if (x == end[0] and y == end[1]):
                    FOUND_DESTINATION = True
                    break
        del Frontier

        if (len(nextFrontier) > 0 and not FOUND_DESTINATION):
            Frontier = nextFrontier.copy()
            del nextFrontier
        else:
            del nextFrontier
            break


    print("Computation Finished")
    parent = end
    path = []
    while True:
        newParent = getParent(parent)
        x, y = (newParent)
        y = int(y)
        x = int(x)


        if (parent[0] == -10):
            print("未找到路径")
            break
        elif ( (parent[0]==x and parent[1]==y)):
            print("路径不存在")
            break

        del parent

        parent=(x,y)
        path.append((x,y))


    print("Total Nodes Checked = ",TotalNodes)
    return path


if __name__=="__main__":
    imgFile = cv2.imread("img_1.jpg")
    
    starting_point = start_point
    ending_point = goal

    backupImg = imgFile.copy()
    graph =extractGraph(imgFile)

    path = BFS(graph,starting_point,ending_point)
    fps = 24
    fourcc = cv2.VideoWriter_fourcc(*'MJPG')  # avi
    video = cv2.VideoWriter(filename='./c.avi', fourcc=fourcc, fps=fps, frameSize=(imgFile.shape[1], imgFile.shape[0]))  
    for point in path:
        point = point[1],point[0]
        cv2.circle(imgFile,point,1,(255,0,0),-1)
        cv2.circle(imgFile,ending_point,3,(255,255,0),-1)
        cv2.circle(imgFile,starting_point,3,(255,0,255),-1)
        video.write(imgFile)
    video.release()

# 视频格式转换

In [ ]:
import cv2
cap = cv2.VideoCapture("c.avi")
videowriter = cv2.VideoWriter("videl_640_480"+".avi", cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'), 24, (640,480))

success, _ = cap.read()

while success:
    success, img1 = cap.read()
    try:
         img = cv2.resize(img1, (640, 480), interpolation=cv2.INTER_LINEAR)
         videowriter.write(img)
    except:
         break

# 解迷宫视频展示

In [ ]:
from pynq.overlays.base import BaseOverlay
from pynq.lib.video import *
base = BaseOverlay("base.bit")
switches = base.switches[1]
# monitor configuration: 640*480 @ 60Hz
Mode = VideoMode(640,480,24)
hdmi_out = base.video.hdmi_out
hdmi_out.configure(Mode,PIXEL_BGR)
hdmi_out.start()
print("HDMI Initialized")

# monitor (output) frame buffer size
frame_out_w = 1600
frame_out_h = 900
# camera (input) configuration
frame_in_w = 640
frame_in_h = 480

# initialize camera from OpenCV
import cv2

videoIn = cv2.VideoCapture('./videl_640_480.avi')
videoIn.set(cv2.CAP_PROP_FRAME_WIDTH, frame_in_w);
videoIn.set(cv2.CAP_PROP_FRAME_HEIGHT, frame_in_h);

print("Capture device is open: " + str(videoIn.isOpened()))

# Capture webcam image

while True:
    ret, frame_vga = videoIn.read()
   
    # display webcam image via HDMI Out
    if (ret):
        outframe = hdmi_out.newframe()
        outframe[0:480,0:640,:] = frame_vga[0:480,0:640,:]
        hdmi_out.writeframe(outframe)
    else:
        raise RuntimeError("Failed to read from camera.")
        break
        
    if switches.read():
        break 
   
videoIn.release()
hdmi_out.stop()
del hdmi_out